In [3]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import json
import os
from tkcalendar import Calendar  # Import Calendar widget for date selection
from datetime import datetime

class TaskManager:
    def __init__(self):
        """Initialize the task manager and load existing tasks."""
        self.tasks = []  # List to store tasks
        self.load_tasks()  # Load tasks from a file at startup

    def load_tasks(self):
        """Load tasks from a JSON file if it exists."""
        if os.path.exists('tasks.json'):  # Check if the file exists
            with open('tasks.json', 'r') as file:  # Open the file in read mode
                self.tasks = json.load(file)  # Load tasks into the list

    def save_tasks(self):
        """Save the current tasks to a JSON file."""
        with open('tasks.json', 'w') as file:  # Open the file in write mode
            json.dump(self.tasks, file)  # Write the tasks to the file in JSON format

    def add_task(self, task_name, category, due_date, priority):
        """Add a new task to the list."""
        task = {
            'name': task_name,  # Task name
            'complete': False,  # Completion status
            'category': category,  # Task category
            'due_date': due_date,  # Due date for the task
            'priority': priority  # Priority level of the task
        }
        self.tasks.append(task)  # Add the task to the list
        self.save_tasks()  # Save the updated task list

    def edit_task(self, task_index, new_name, category, due_date, priority):
        """Edit an existing task's details."""
        if 0 <= task_index < len(self.tasks):  # Check if the index is valid
            self.tasks[task_index]['name'] = new_name  # Update task name
            self.tasks[task_index]['category'] = category  # Update category
            self.tasks[task_index]['due_date'] = due_date  # Update due date
            self.tasks[task_index]['priority'] = priority  # Update priority
            self.save_tasks()  # Save changes
            return True  # Indicate success
        return False  # Indicate failure

    def delete_task(self, task_index):
        """Delete a task from the list."""
        if 0 <= task_index < len(self.tasks):  # Check if the index is valid
            self.tasks.pop(task_index)  # Remove the task from the list
            self.save_tasks()  # Save changes
            return True  # Indicate success
        return False  # Indicate failure

    def mark_task_complete(self, task_index):
        """Mark a task as complete."""
        if 0 <= task_index < len(self.tasks):  # Check if the index is valid
            self.tasks[task_index]['complete'] = True  # Set completion status
            self.save_tasks()  # Save changes
            return True  # Indicate success
        return False  # Indicate failure

    def get_tasks(self):
        """Return the current list of tasks."""
        return self.tasks  # Return the task list

class App:
    def __init__(self, root):
        """Initialize the main application window."""
        self.task_manager = TaskManager()  # Create an instance of TaskManager
        self.root = root
        self.root.title("Task Manager")  # Set the title of the window

        # Frame for the task list
        self.frame = tk.Frame(self.root)
        self.frame.pack(pady=10)  # Add padding around the frame

        # Listbox to display tasks
        self.task_listbox = tk.Listbox(self.frame, width=70, height=15)
        self.task_listbox.pack(side=tk.LEFT)  # Place listbox on the left

        # Scrollbar for the task list
        self.scrollbar = tk.Scrollbar(self.frame)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)  # Fill vertically

        # Configure scrollbar and listbox
        self.task_listbox.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.task_listbox.yview)

        # Entry for new tasks
        self.task_name_entry = tk.Entry(self.root, width=52)
        self.task_name_entry.pack(pady=10)  # Add padding

        # Combobox for selecting task categories
        self.category_var = tk.StringVar(self.root)
        self.category_combobox = ttk.Combobox(self.root, textvariable=self.category_var,
            values=["Personal", "Work", "School", "Fitness", "Other"], state="readonly")
        self.category_combobox.set("Select Category")  # Default value
        self.category_combobox.pack(pady=10)  # Add padding

        # Button to select due date using a calendar
        self.due_date_button = tk.Button(self.root, text="Select Due Date", command=self.open_calendar)
        self.due_date_button.pack(pady=10)
        self.due_date = None  # Variable to store selected due date

        # Combobox for selecting task priority
        self.priority_var = tk.StringVar(self.root)
        self.priority_combobox = ttk.Combobox(self.root, textvariable=self.priority_var,
            values=["1 - High", "2 - Medium", "3 - Low"], state="readonly")
        self.priority_combobox.set("Select Priority")  # Default value
        self.priority_combobox.pack(pady=10)

        # Buttons for task management functionalities
        self.add_task_button = tk.Button(self.root, text="Add Task", command=self.add_task)
        self.add_task_button.pack(pady=5)

        self.edit_task_button = tk.Button(self.root, text="Edit Task", command=self.edit_task)
        self.edit_task_button.pack(pady=5)

        self.delete_task_button = tk.Button(self.root, text="Delete Task", command=self.delete_task)
        self.delete_task_button.pack(pady=5)

        self.mark_complete_button = tk.Button(self.root, text="Mark Task Complete", command=self.mark_task_complete)
        self.mark_complete_button.pack(pady=5)

        self.populate_tasks()  # Load existing tasks into the listbox

    def open_calendar(self):
        """Open a calendar popup to select the due date."""
        top = tk.Toplevel(self.root)  # Create a new top-level window
        top.title("Select Due Date")  # Set the title of the calendar window
        
        # Create a calendar widget
        self.calendar = Calendar(top, selectmode='day', year=datetime.now().year,
                                 month=datetime.now().month, day=datetime.now().day)
        self.calendar.pack(pady=20)  # Add padding
        
        # Button to select the date from the calendar
        select_button = tk.Button(top, text="Select", command=lambda: self.set_due_date(top))
        select_button.pack(pady=10)

    def set_due_date(self, calendar_window):
        """Set the selected date as the due date and close the calendar window."""
        self.due_date = self.calendar.get_date()  # Get the selected date
        calendar_window.destroy()  # Close the calendar window

    def populate_tasks(self):
        """Populate the listbox with tasks from the task manager."""
        self.task_listbox.delete(0, tk.END)  # Clear existing tasks in the listbox
        for task in self.task_manager.get_tasks():  # Iterate through tasks
            # Determine the status and create a display string for each task
            status = "✔️ Complete" if task['complete'] else "❌ Incomplete"
            display_text = f"{task['name']} - {status} | Due: {task['due_date']} | Category: {task['category']} | Priority: {task['priority']}"
            self.task_listbox.insert(tk.END, display_text)  # Add task to the listbox

    def add_task(self):
        """Add a new task from the entry fields."""
        task_name = self.task_name_entry.get()  # Get the task name from entry
        category = self.category_var.get()  # Get selected category
        due_date = self.due_date  # Get the selected due date
        priority = self.priority_var.get()  # Get selected priority

        # Check if task name, category, due date, and priority are valid
        if not task_name or category == "Select Category" or not due_date or priority == "Select Priority":
            messagebox.showwarning("Input Error", "Please fill in all fields.")
            return  # Exit the method if validation fails

        # Convert priority to numeric value
        priority_value = int(priority[0])  # Get the numeric part of the priority
        self.task_manager.add_task(task_name, category, due_date, priority_value)  # Add the task
        self.populate_tasks()  # Refresh the task list display
        self.clear_entries()  # Clear the input fields

    def edit_task(self):
        """Edit the selected task."""
        try:
            selected_index = self.task_listbox.curselection()[0]  # Get the index of the selected task
            new_name = self.task_name_entry.get()  # Get the new task name
            category = self.category_var.get()  # Get the new category
            due_date = self.due_date  # Get the new due date
            priority = self.priority_var.get()  # Get the new priority
            
            if not due_date:  # Check if due date is selected
                messagebox.showwarning("Input Error", "Please select a due date.")
                return  # Exit if validation fails
            
            # Convert priority to numeric value
            priority_value = int(priority[0])  # Get the numeric part of the priority
            if self.task_manager.edit_task(selected_index, new_name, category, due_date, priority_value):  # Edit the task
                self.populate_tasks()  # Refresh the task list
                self.clear_entries()  # Clear the input fields
            else:
                messagebox.showerror("Edit Error", "Failed to edit task.")
        except IndexError:
            messagebox.showwarning("Selection Error", "Please select a task to edit.")  # Handle no selection

    def delete_task(self):
        """Delete the selected task."""
        try:
            selected_index = self.task_listbox.curselection()[0]  # Get the index of the selected task
            if self.task_manager.delete_task(selected_index):  # Delete the task
                self.populate_tasks()  # Refresh the task list
            else:
                messagebox.showerror("Delete Error", "Failed to delete task.")
        except IndexError:
            messagebox.showwarning("Selection Error", "Please select a task to delete.")  # Handle no selection

    def mark_task_complete(self):
        """Mark the selected task as complete."""
        try:
            selected_index = self.task_listbox.curselection()[0]  # Get the index of the selected task
            if self.task_manager.mark_task_complete(selected_index):  # Mark task as complete
                self.populate_tasks()  # Refresh the task list
            else:
                messagebox.showerror("Completion Error", "Failed to mark task complete.")
        except IndexError:
            messagebox.showwarning("Selection Error", "Please select a task to mark as complete.")  # Handle no selection

    def clear_entries(self):
        """Clear the input fields."""
        self.task_name_entry.delete(0, tk.END)  # Clear task name entry
        self.category_var.set("Select Category")  # Reset category combobox
        self.priority_var.set("Select Priority")  # Reset priority combobox
        self.due_date = None  # Clear due date

if __name__ == "__main__":
    root = tk.Tk()  # Create the main application window
    app = App(root)  # Initialize the app
    root.mainloop()  # Start the Tkinter event loop
